In [27]:
pip install yfinance pandas

Note: you may need to restart the kernel to use updated packages.


In [28]:
# --- 1. Import Necessary Libraries ---
# yfinance and pandas are pre-installed on Kaggle
import yfinance as yf
import pandas as pd
import os

# Define the stock ticker and the time period
TICKER_SYMBOL = "RELIANCE.NS"
START_DATE = "2015-01-01"
END_DATE = "2025-08-25" 

# Define the output file path within the Kaggle working directory
OUTPUT_FILE = f"/kaggle/working/{TICKER_SYMBOL}_historical.csv"

print(f"Downloading historical data for {TICKER_SYMBOL} from {START_DATE} to {END_DATE}...")

try:
    # Use the yf.download function to get the data
    df = yf.download(TICKER_SYMBOL, start=START_DATE, end=END_DATE)

    if df.empty:
        print("No data was downloaded. Please check the ticker symbol and date range.")
    else:
        # Save the data to a CSV file in the Kaggle working directory
        df.to_csv(OUTPUT_FILE)
        print(f"Successfully downloaded and saved data to {OUTPUT_FILE}")
        print("\nFirst 5 rows of the downloaded data:")
        print(df.head())

except Exception as e:
    print(f"An error occurred: {e}")

/tmp/ipykernel_36/3209400723.py:19: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(TICKER_SYMBOL, start=START_DATE, end=END_DATE)
[*********************100%***********************]  1 of 1 completed

Successfully downloaded and saved data to /kaggle/working/RELIANCE.NS_historical.csv

First 5 rows of the downloaded data:
Price            Close        High         Low        Open      Volume
Ticker     RELIANCE.NS RELIANCE.NS RELIANCE.NS RELIANCE.NS RELIANCE.NS
Date                                                                  
2015-01-01  189.999786  190.877140  189.090335  189.657416     2963643
2015-01-02  189.496948  191.743828  189.229471  190.042616     7331366
2015-01-05  187.421265  190.641790  187.046782  189.379255    10103941
2015-01-06  178.915237  186.811392  178.037884  186.169433    18627980
2015-01-07  182.809830  183.772776  179.107845  179.129243    20720312


In [29]:
# This line installs the pandas-ta library inside the Kaggle notebook
!pip install pandas-ta

In [30]:
# --- 1. Import Necessary Libraries ---
import pandas as pd
import pandas_ta as ta

# --- 2. Load the Data from the Previous Step ---
# This code correctly handles the CSV file format from yfinance.
try:
    # Use index_col=0 to tell pandas that the first column is the date.
    df = pd.read_csv(
        '/kaggle/working/RELIANCE.NS_historical.csv',
        index_col=0,  # The first column (index 0) is the date
        parse_dates=True
    )
    # The index is a datetime object, but it doesn't have a name. Let's give it one.
    df.index.name = 'Date'

    # Now, let's remove the extra ticker column that might be in the DataFrame
    # This is a defensive programming practice.
    if 'RELIANCE.NS' in df.columns:
        df = df.drop(columns=['RELIANCE.NS'])
        
    # --- Check for non-numeric columns and convert them ---
    # This is the most important part to fix your error.
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    print("Data loaded successfully for strategy development.")
    print("DataFrame dtypes after conversion:")
    print(df.dtypes)
    print("\nDataFrame head:")
    print(df.head())
    
except FileNotFoundError:
    print("Error: Data file not found. Please ensure your previous step completed successfully.")
    raise SystemExit

# --- 3. Create Technical Indicators ---
# This code will now work because all columns are numeric.
df['SMA_20'] = ta.sma(df['Close'], length=20)
df['SMA_50'] = ta.sma(df['Close'], length=50)

# We can also add other indicators for future use, like RSI
df['RSI'] = ta.rsi(df['Close'], length=14)

print("\nFirst 10 rows with new indicators:")
print(df.tail(10))

# --- 4. Handle NaN values ---
df.dropna(inplace=True)
print(f"\nDataFrame shape after dropping NaNs: {df.shape}")

# --- 5. Define the Trading Signals ---
df['Signal'] = 0
df.loc[df['SMA_20'] > df['SMA_50'], 'Signal'] = 1
df.loc[df['SMA_20'] < df['SMA_50'], 'Signal'] = -1

df['Position'] = df['Signal'].diff()

print("\nFinal DataFrame with Trading Signals:")
print(df.tail())

Data loaded successfully for strategy development.
DataFrame dtypes after conversion:
Close     float64
High      float64
Low       float64
Open      float64
Volume    float64
dtype: object

DataFrame head:
                 Close        High         Low        Open      Volume
Date                                                                  
Ticker             NaN         NaN         NaN         NaN         NaN
Date               NaN         NaN         NaN         NaN         NaN
2015-01-01  189.999786  190.877140  189.090335  189.657416   2963643.0
2015-01-02  189.496948  191.743828  189.229471  190.042616   7331366.0
2015-01-05  187.421265  190.641790  187.046782  189.379255  10103941.0

First 10 rows with new indicators:
                  Close         High          Low         Open      Volume  \
Date                                                                         
2025-08-08  1362.358887  1391.641835  1359.569977  1379.490416   8793606.0   
2025-08-11  1380.685669  1

/tmp/ipykernel_36/999834622.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)
/usr/local/lib/python3.11/dis

In [31]:
# This installs the backtesting.py library inside your Kaggle notebook
!pip install backtesting

In [36]:
# --- 1. Load the Data from the Previous Step ---
import pandas as pd
import io

try:
    # Read the data manually into a list of lists to handle column mismatch
    with open('/kaggle/working/RELIANCE.NS_historical.csv', 'r') as f:
        data_rows = f.readlines()

    # Find the header row (it usually contains 'Date')
    header_index = 0
    for i, row in enumerate(data_rows):
        if 'Date' in row or 'Open' in row:
            header_index = i
            break
    
    # Read the data from the header row onwards
    df = pd.read_csv(
        io.StringIO(''.join(data_rows[header_index:])),
        index_col=0,
        parse_dates=True
    )
    print("DataFrame successfully loaded!")
    print(df.head())
    print("\nDataFrame Info:")
    df.info()

except FileNotFoundError:
    print("Error: Data file not found.")
    raise SystemExit

DataFrame successfully loaded!
                         Close                High                 Low  \
Price                                                                    
Ticker             RELIANCE.NS         RELIANCE.NS         RELIANCE.NS   
Date                       NaN                 NaN                 NaN   
2015-01-01  189.99978637695312   190.8771400134829  189.09033548185477   
2015-01-02   189.4969482421875  191.74382815812206  189.22947104058622   
2015-01-05   187.4212646484375   190.6417900892634  187.04678228843514   

                          Open       Volume  
Price                                        
Ticker             RELIANCE.NS  RELIANCE.NS  
Date                       NaN          NaN  
2015-01-01  189.65741561888825      2963643  
2015-01-02  190.04261601795088      7331366  
2015-01-05  189.37925486692558     10103941  

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
Index: 2630 entries, Ticker to 2025-08-22
Data columns (total 5 columns):

/tmp/ipykernel_36/540105279.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(


In [38]:
# --- 1. Load the Data from the Previous Step ---
import pandas as pd
import io

try:
    # Read the data manually into a list of lists to handle column mismatch
    with open('/kaggle/working/RELIANCE.NS_historical.csv', 'r') as f:
        data_rows = f.readlines()

    # Find the header row (it usually contains 'Date')
    header_index = 0
    for i, row in enumerate(data_rows):
        if 'Date' in row or 'Open' in row:
            header_index = i
            break
    
    # Read the data from the header row onwards
    df = pd.read_csv(
        io.StringIO(''.join(data_rows[header_index:])),
        index_col=0,
        parse_dates=True
    )
    print("DataFrame successfully loaded!")
    print(df.head())
    print("\nDataFrame Info:")
    df.info()

except FileNotFoundError:
    print("Error: Data file not found.")
    raise SystemExit

DataFrame successfully loaded!
                         Close                High                 Low  \
Price                                                                    
Ticker             RELIANCE.NS         RELIANCE.NS         RELIANCE.NS   
Date                       NaN                 NaN                 NaN   
2015-01-01  189.99978637695312   190.8771400134829  189.09033548185477   
2015-01-02   189.4969482421875  191.74382815812206  189.22947104058622   
2015-01-05   187.4212646484375   190.6417900892634  187.04678228843514   

                          Open       Volume  
Price                                        
Ticker             RELIANCE.NS  RELIANCE.NS  
Date                       NaN          NaN  
2015-01-01  189.65741561888825      2963643  
2015-01-02  190.04261601795088      7331366  
2015-01-05  189.37925486692558     10103941  

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
Index: 2630 entries, Ticker to 2025-08-22
Data columns (total 5 columns):

/tmp/ipykernel_36/540105279.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(


In [49]:
# --- 1. All Necessary Imports ---
import pandas as pd
import pandas_ta as ta
from backtesting import Strategy, Backtest
from backtesting.lib import crossover
import numpy as np
import io

# --- 2. Load the Data with a Robust Method ---
try:
    # Read the data manually into a list of lists to handle column mismatch
    with open('/kaggle/working/RELIANCE.NS_historical.csv', 'r') as f:
        data_rows = f.readlines()

    # Find the header row (it usually contains 'Date')
    header_index = 0
    for i, row in enumerate(data_rows):
        if 'Date' in row or 'Open' in row:
            header_index = i
            break
    
    # Read the data from the header row onwards
    df = pd.read_csv(
        io.StringIO(''.join(data_rows[header_index:])),
        index_col=0,
        parse_dates=True
    )
    
except FileNotFoundError:
    raise SystemExit("Data file not found.")

# --- 3. Clean and Prepare the Data ---
# Convert all columns to numeric, forcing errors to NaN
for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Drop any rows with NaN values
df.dropna(inplace=True)

# --- 4. Create the Technical Indicators ---
df['sma_20'] = ta.sma(df['Close'], length=20)
df['sma_50'] = ta.sma(df['Close'], length=50)

# Drop rows where SMA values are NaN (beginning of dataset)
df.dropna(inplace=True)

# --- 5. Prepare Data for Backtesting ---
df_backtest = df.copy()
df_backtest.columns = [col.capitalize() for col in df_backtest.columns]

# Make sure the index is a proper datetime index
if not isinstance(df_backtest.index, pd.DatetimeIndex):
    try:
        df_backtest.index = pd.to_datetime(df_backtest.index)
    except:
        # If we can't convert to datetime, create a simple range index
        df_backtest.index = pd.RangeIndex(start=0, stop=len(df_backtest))

# --- 6. Define the Trading Strategy ---
class SmaCross(Strategy):
    n1 = 20
    n2 = 50

    def init(self):
        # Precompute the SMAs
        close = pd.Series(self.data.Close)
        self.sma1 = self.I(ta.sma, close, self.n1, name='SMA_20')
        self.sma2 = self.I(ta.sma, close, self.n2, name='SMA_50')

    def next(self):
        if crossover(self.sma1, self.sma2):
            if not self.position:
                self.buy()
        elif crossover(self.sma2, self.sma1):
            if self.position:
                self.sell()

# --- 7. Run the Backtest ---
bt = Backtest(df_backtest, SmaCross, cash=100000, commission=.002, 
              exclusive_orders=True, trade_on_close=True)

stats = bt.run()

print("\nBacktest Results:")
print(stats)

# --- 8. Plot the Results ---
bt.plot()

/tmp/ipykernel_36/426589129.py:23: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(


Backtest.run:   0%|          | 0/2529 [00:00<?, ?bar/s]

/tmp/ipykernel_36/426589129.py:82: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  stats = bt.run()



Backtest Results:
Start                     2015-03-16 00:00:00
End                       2025-08-22 00:00:00
Duration                   3812 days 00:00:00
Exposure Time [%]                    93.64095
Equity Final [$]                   6378.24746
Equity Peak [$]                  112526.72562
Commissions [$]                    3611.23136
Return [%]                          -93.62175
Buy & Hold Return [%]                642.5761
Return (Ann.) [%]                   -23.58042
Volatility (Ann.) [%]                23.58833
CAGR [%]                            -16.63527
Sharpe Ratio                         -0.99966
Sortino Ratio                        -1.04694
Calmar Ratio                         -0.24817
Alpha [%]                           502.86367
Beta                                 -0.92827
Max. Drawdown [%]                   -95.01726
Avg. Drawdown [%]                   -14.59689
Max. Drawdown Duration     3509 days 00:00:00
Avg. Drawdown Duration      449 days 00:00:00
# Trades       

GridPlot(id='p1349', ...)

In [50]:
# Create a dictionary of your backtest results
# These are the values you got from the 'stats' variable
performance_metrics = {
    'Metric': [
        'Total Return (%)',
        'Sharpe Ratio',
        'Max Drawdown (%)',
        'Win Rate (%)'
    ],
    'Value': [
        stats['Return [%]'],
        stats['Sharpe Ratio'],
        stats['Max. Drawdown [%]'],
        stats['Win Rate [%]']
    ]
}

results_df = pd.DataFrame(performance_metrics)

print("--- QuantFlow Backtest Performance ---")
print(results_df.round(2))

--- QuantFlow Backtest Performance ---
             Metric  Value
0  Total Return (%) -93.62
1      Sharpe Ratio  -1.00
2  Max Drawdown (%) -95.02
3      Win Rate (%)  22.58
